In [35]:
pip install transformers datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
import torch
if torch.cuda.is_available():
    print("CUDA is available. GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

CUDA is available. GPU: Tesla T4


In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [45]:
import os
import time
from git import Repo
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, concatenate_datasets

In [39]:
# Step 3: Load the HumanEval dataset
# print("Loading the HumanEval dataset...")
# dataset = load_dataset("openai_humaneval")

# Load the HumanEval-X dataset for all languages
print("Loading the HumanEval-X datasets...")
humaneval_x_dataset_py = load_dataset("THUDM/humaneval-x", "python")
humaneval_x_dataset_cpp = load_dataset("THUDM/humaneval-x", "cpp")
humaneval_x_dataset_go = load_dataset("THUDM/humaneval-x", "go")
humaneval_x_dataset_java = load_dataset("THUDM/humaneval-x", "java")
humaneval_x_dataset_js = load_dataset("THUDM/humaneval-x", "js")
# humaneval_x_dataset_rust = load_dataset("THUDM/humaneval-x", "rust")

# Combine the datasets
print("Combining dataset...")
combined_dataset = concatenate_datasets([  humaneval_x_dataset_py["test"], humaneval_x_dataset_cpp["test"],
                                         humaneval_x_dataset_go["test"],humaneval_x_dataset_java["test"] ,
                                         humaneval_x_dataset_js["test"]])


Loading the HumanEval dataset...
Loading the HumanEval-X datasets...
Combining dataset...


In [40]:
print(humaneval_x_dataset_py)
print(combined_dataset)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'declaration', 'canonical_solution', 'test', 'example_test'],
        num_rows: 164
    })
})
Dataset({
    features: ['task_id', 'prompt', 'declaration', 'canonical_solution', 'test', 'example_test'],
    num_rows: 820
})


In [41]:
model_name = "Salesforce/codegen-350M-mono"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


In [42]:
import re

def generate_code(prompt, model, tokenizer, device, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, num_return_sequences=1)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Post-process to keep only the main function and ensure it ends properly
    if "def " in generated_code:
        # Extract the main function including everything after it
        main_function = generated_code.split("def ", 1)[1]
        main_function = "def " + main_function  # Re-add the def keyword
        
        # Detect end of function by finding the next function or end of indentation
        lines = main_function.split('\n')
        function_lines = []
        inside_function = False
        indentation_level = None
        
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("def ") and inside_function:
                break  # New function starts, stop collecting lines
            if not inside_function:
                if stripped_line.startswith("def "):
                    inside_function = True
                    indentation_level = len(line) - len(line.lstrip())
            if inside_function:
                if stripped_line == "" or line.startswith(" " * indentation_level):
                    function_lines.append(line)
                else:
                    break  # End of the function when the indentation changes

        complete_function = "\n".join(function_lines)
        return complete_function
    return generated_code




In [49]:
run_duration = int(input("Enter the duration to run the script (in seconds): "))

Enter the duration to run the script (in seconds):  4000


In [ ]:
# Step 5: Generate code solutions
# repeat this to include how much time to run this : change timing over here to run for longer. 
print("Generating code solutions...")
run_duration=3600
prompts = [sample["prompt"] for sample in combined_dataset]
generated_solutions = []

start_time = time.time()
current_time=0

while current_time < run_duration:
    for prompt in prompts:
        current_time = time.time() - start_time
        if current_time >= run_duration:
            break
        solution = generate_code(prompt, model, tokenizer, device)
        generated_solutions.append(solution)
        print(f"Prompt:\n{prompt}")
        print(f"Generated Solution:\n{solution}\n")

print(f"Completed processing. Generated solutions for {len(generated_solutions)} prompts in {run_duration} seconds.")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating code solutions...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

Generated Solution:
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers) - 1):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt:


def sort_even(l: list):
    """This function takes a list l and returns a list l' such that
    l' is identical to l in the odd indicies, while its values at the even indicies are equal
    to the values of the even indicies of l, but sorted.
    >>> sort_even([1, 2, 3])
    [1, 2, 3]
    >>> sort_even([5, 6, 3, 4])
    [3, 6, 5, 4]
    """

Generated Solution:
def sort_even(l: list):
    """This function takes a list l and returns a list l' such that
    l' is identical to l in the odd indicies, while its values at the even indicies are equal
    to the values of the even indicies of l, but sorted.
    >>> sort_even([1, 2, 3])
    [1, 2, 3]
    >>> sort_even([5, 6, 3, 4])
    [3, 6, 5, 4]
    """
    even_indices = []
    odd_indices = []
    for i in range(len(l)):
        if i % 2 == 0:
            even_indices.append(l[i])
        else:
            odd_indices.append(l[i])
    return even_indices + odd_indices




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt:


def encode_cyclic(s: str):
    """
    returns encoded string by cycling groups of three characters.
    """
    # split string to groups. Each of length 3.
    groups = [s[(3 * i):min((3 * i + 3), len(s))] for i in range((len(s) + 2) // 3)]
    # cycle elements in each group. Unless group has fewer elements than 3.
    groups = [(group[1:] + group[0]) if len(group) == 3 else group for group in groups]
    return "".join(groups)


def decode_cyclic(s: str):
    """
    takes as input string encoded with encode_cyclic function. Returns decoded string.
    """

Generated Solution:
def encode_cyclic(s: str):
    """
    returns encoded string by cycling groups of three characters.
    """
    # split string to groups. Each of length 3.
    groups = [s[(3 * i):min((3 * i + 3), len(s))] for i in range((len(s) + 2) // 3)]
    # cycle elements in each group. Unless group has fewer elements than 3.
    groups = [(group[1:] + group[0]) if len(group) == 3 else group for group in groups

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
